#과제

##Titanic 데이터를 가지고 5 개의 FE 방법들 직접 사용해보기 (오늘 피피티에 소개되지 않은 FE 방법도 괜찮음)

In [94]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [96]:
data = pd.read_csv('/content/gdrive/MyDrive/titanic.csv')

In [97]:
data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [98]:
data.info()
data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

# 결측값 대체 (Imputing)

In [101]:
data_2 = data.fillna(0)
print("결측치 대체 전 데이터 개수 : ", data.shape[0])
print("결측치 대체 후 데이터 개수 : ", data_2.shape[0])

결측치 대체 전 데이터 개수 :  418
결측치 대체 후 데이터 개수 :  418


In [102]:
####결측치를 Mean or Median으로 대체####
data_3 = data.fillna(data.mean())
data_4 = data.fillna(data.median())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  This is separate from the ipykernel package so we can avoid doing imports until


In [103]:
####결측치를 가장 빈도수가 높은 값으로 대체####
print(data['Age'].value_counts())
print(data['Fare'].value_counts())
print(data['Cabin'].value_counts())

21.0    17
24.0    17
22.0    16
30.0    15
18.0    13
        ..
76.0     1
28.5     1
22.5     1
62.0     1
38.5     1
Name: Age, Length: 79, dtype: int64
7.7500     21
26.0000    19
13.0000    17
8.0500     17
7.8958     11
           ..
7.8208      1
8.5167      1
78.8500     1
52.0000     1
22.3583     1
Name: Fare, Length: 169, dtype: int64
B57 B59 B63 B66    3
B45                2
C89                2
C55 C57            2
A34                2
                  ..
E52                1
D30                1
E31                1
C62 C64            1
C105               1
Name: Cabin, Length: 76, dtype: int64


In [104]:
# 빈도수가 높은 값으로 채워줌
data_5 = data.copy()
data_5.loc[data['Age'] != data['Age'],'Age'] = 21.0
data_5.loc[data['Fare'] != data['Fare'],'Fare'] = 7.7500
data_5.loc[data['Cabin'] != data['Cabin'],'Cabin'] = 'B57'
print("결측값 대체 전 Age의 21.0 값 개수 : ", data.Age.value_counts()[21.0])
print("결측값 대체 후 Age의 21.0 값 개수 : ", data_5.Age.value_counts()[21.0])
print("결측값 대체 전 Fare의 7.7500 값 개수 : ", data.Fare.value_counts()[7.7500])
print("결측값 대체 후 Fare의 7.7500 값 개수 : ", data_5.Fare.value_counts()[7.7500])
print("결측값 대체 전 Cabin의 B57 값 개수 : ", data.Cabin.value_counts()['B57 B59 B63 B66'])
print("결측값 대체 후 Cabin의 B57 값 개수 : ", data_5.Cabin.value_counts()['B57'])

결측값 대체 전 Age의 21.0 값 개수 :  17
결측값 대체 후 Age의 21.0 값 개수 :  103
결측값 대체 전 Fare의 7.7500 값 개수 :  21
결측값 대체 후 Fare의 7.7500 값 개수 :  22
결측값 대체 전 Cabin의 B57 값 개수 :  3
결측값 대체 후 Cabin의 B57 값 개수 :  327


In [105]:
####결측치를 임의의 값으로 대체####
data_6 = data.fillna(method = 'pad')
data_7 = data.fillna(method = 'bfill')

# 지표변수 (Indicator variables)

In [106]:
data['Pclass'].unique()

array([3, 2, 1])

In [107]:
classes = []
for i in data['Pclass']:
    if (i == 1):
        classes.append('first-class')
    elif (i == 2):
        classes.append('second-class')
    else:
        classes.append('third-class')

data['Classes'] = classes

In [108]:
data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Classes
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,third-class
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,third-class
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,second-class
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,third-class
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,third-class


## Feature Split

In [109]:
for i in data:
    data['Title'] = data.Name.str.extract('([A-Za-z]+)\.')
    data['Fisrt-Name'] = data.Name.str.extract('([A-Za-z]+)\,')

In [110]:
data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Classes,Title,Fisrt-Name
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,third-class,Mr,Kelly
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,third-class,Mrs,Wilkes
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,second-class,Mr,Myles
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,third-class,Mr,Wirz
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,third-class,Mrs,Hirvonen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,third-class,Mr,Spector
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,first-class,Dona,Ocana
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,third-class,Mr,Saether
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,third-class,Mr,Ware


## 스케일링 Scaling

In [111]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
mm_scaler = MinMaxScaler()
sd_scaler = StandardScaler()

In [112]:
data_1 = data.fillna(0)

# MinMax Scaling
mm_scaler = mm_scaler.fit_transform(data_1['Fare'].values.reshape(-1,1))
data['mm_scaled_Fare'] = mm_scaler

# Standard Scaling
sd_scaler = sd_scaler.fit_transform(data_1['Fare'].values.reshape(-1,1))
data['sd_scaled_Fare'] = sd_scaler


In [ ]:
data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Classes,Title,Fisrt-Name,mm_scaled_Fare,sd_scaled_Fare
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,third-class,Mr,Kelly,0.015282,-0.496637
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,third-class,Mrs,Wilkes,0.013663,-0.511497
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,second-class,Mr,Myles,0.018909,-0.463335
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,third-class,Mr,Wirz,0.016908,-0.481704
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,third-class,Mrs,Hirvonen,0.023984,-0.416740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,third-class,Mr,Spector,0.015713,-0.492680
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,first-class,Dona,Ocana,0.212559,1.314641
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,third-class,Mr,Saether,0.014151,-0.507017
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,third-class,Mr,Ware,0.015713,-0.492680


## 라벨 인코딩 (Label Encoding)

In [113]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [114]:
encoded = encoder.fit_transform(data['Embarked'])

In [115]:
data['Embarked_class'] = encoded

In [116]:
data.Embarked_class.value_counts()

2    270
0    102
1     46
Name: Embarked_class, dtype: int64

In [117]:
data

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Classes,Title,Fisrt-Name,mm_scaled_Fare,sd_scaled_Fare,Embarked_class
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,third-class,Mr,Kelly,0.015282,-0.496637,1
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,third-class,Mrs,Wilkes,0.013663,-0.511497,2
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,second-class,Mr,Myles,0.018909,-0.463335,1
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,third-class,Mr,Wirz,0.016908,-0.481704,2
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,third-class,Mrs,Hirvonen,0.023984,-0.416740,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S,third-class,Mr,Spector,0.015713,-0.492680,2
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,first-class,Dona,Ocana,0.212559,1.314641,0
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,third-class,Mr,Saether,0.014151,-0.507017,2
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S,third-class,Mr,Ware,0.015713,-0.492680,2


## 원-핫 인코딩 (One-Hot Encoding)

In [118]:
pd.get_dummies(data['Embarked'])

,C,Q,S
0,0,1,0
1,0,0,1
2,0,1,0
3,0,0,1
4,0,0,1
...,...,...,...
413,0,0,1
414,1,0,0
415,0,0,1
416,0,0,1
